In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Bidirectional, Dropout
from keras.layers import LSTM, GRU
from keras.optimizers import RMSprop, Adam

C:\Users\bkrak_000\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9391772275352634356
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4984510873
locality {
  bus_id: 1
}
incarnation: 15875972506067057396
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [92]:
p = re.compile(r"\d")

with open("C:/Users/bkrak_000/Downloads/plurality.txt", encoding='utf-8', errors='replace') as f:
    text = f.read().lower()
    text = p.sub('', text)
print ("corpus length = {}".format(len(text)))
    

corpus length = 779966


In [93]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 48


In [94]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
# sentences = overlapping sequence of characters of len 40
sentences = []
# next_chars = the next character for every sentence
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 259976


In [95]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [12]:
# build the model: a single GRU
print('Build model...')
model = Sequential()
model.add(Bidirectional(GRU(128), input_shape=(maxlen, len(chars))))
model.add(Dropout(.3))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.001, clipnorm = 1.)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [96]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) #doin' some softmaxing?
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#why not do this with np.random.choice over a probability distribution?

In [97]:
import random
import sys

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [105]:
### from keras.callbacks import LambdaCallback


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=200,
          callbacks=[print_callback])

Epoch 1/200
259976/259976 [==============================] - 153s 589us/step - loss: 0.8227

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ed in this relative way could not be the"
ed in this relative way could not be the negative world. 
about the not an even the relation of the one world. but i do not be a doxastic alternatives or what we say in the same world where there is a counterpart in the thing really is an and analogic else some world. the concrete world is the same thing, the same thing may be a set of set of the same plan to say if the trope is one that is that there is no case. but the thing some sort
----- diversity: 0.5
----- Generating with seed: "ed in this relative way could not be the"
ed in this relative way could not be the form of the trans-world individuals that there is the third world. it is a many way that modal realism at a possibility of property of reason be a relations and the nature of the two representation de re;

tents of thought because they are utterless of the forme. if so, it is 'not true in a possibility of sentences and the ersatz world is a contradiction. in this world has no single the simplicity in the set of a set of it doesn't see has the and nevertheless the sense if such there are non- on the content of the such doneis me misesely set to the content of the more forterself hoom of the properties. thereby have any particular individua
----- diversity: 0.5
----- Generating with seed: "tents of thought because they are utterl"
tents of thought because they are utterless of the charges carnal location of the plastic in counterpart and arbitral atomic but fall and that sort of include the problem would be enternally, is the way the problem of it. 
i think that such sentences that me greaters to the controversial relations and problem of reason. and it would have been to seated believe in a given of the predications and sentences modal realism. 
there is no diff
----- diversity: 1.0
-----

tion holds throughout your doxastically accessible worlds. the concrete world is that the concrete world is that there is a counterpart of a world of the things that worlds where the many one something can be said. the concrete world ersatz worlds might say is the distance in the simpler world, i take the properties are the ones that the any different worlds, and it is not the concrete world. () use ce ceptere, or is not include in the 
----- diversity: 0.5
----- Generating with seed: "tion holds throughout your doxastically "
tion holds throughout your doxastically accessible worlds something has to do with it is incounterported to distinguish the single external relations of similarity that as only that world w is on what they really a world for the things that he should be problem for any relation to suppose who with useful space that they are of aspect worlds is it is not that world. the concrete world does not accept the particle to want to be four set t
----- diversity: 1.0
-----

the such matter of things to be modal realism. there is no donkey talks in the sense in the properties. there is no of any world where it worlds have the parallel futures. still i do not say that if there are more surestic out to be the content of the first theoretical realisms. 
suppose on the planet to be specify the possibilities are not both o
----- diversity: 0.5
----- Generating with seed: "er i am the second-born. but they are no"
er i am the second-born. but they are not say that some world is true that is true. i cannot any concerted proposition that picture is the properties. 
the other worlds on other particles and our world. there is no of which we do not exists according to a certain way that if we not be a question has a counterpart relation of the insister shaped. this things such that there are a description of the borders. there is a given actual world 
----- diversity: 1.0
----- Generating with seed: "er i am the second-born. but they are no"
er i am the second-born. 

. not, at any rate, here in what the ersatz worlds are tropes to all the nature if there that one of the other-worldly and as any given many of the modal on which such a world we have in which way that the worlds with a part of a world where the pictures will have a part of any possibility. 
i have donkey talkent for unify the concrete world be properties. 
() is the even so and exactly can part when i have in concernar semantic value p
----- diversity: 1.0
----- Generating with seed: ". not, at any rate, here in what the ers"
. not, at any rate, here in what the ersatz worlds cuppres. equivalents thought in how p of our  different of plastic names actually intrinsic tropes at all with any quantification over corresponding controversial, and believe in a world does not differ an any world, with a wast that was a negator we need- one happens to the same syntrovils, we could say that the one is sole of the space-terminaal andablumesters are diemence partly, i c
----- diversity: 1.2
-----

...

259976/259976 [==============================] - 143s 551us/step - loss: 0.8143

----- Generating text after Epoch: 170
----- diversity: 0.2
----- Generating with seed: "r word the same. (and countless more tim"
r word the same. (and countless more times, and as in the set of any part of a world we is a world where the concrete world is instantiated. 
plain pary the speaker common part of a part of a world composed of parts of worlds a will a world we have some worlds in simply which worlds are all of the concrete world, and all it, the ersatzer there are other worlds that are part of a world where the concrete world is instantiated by a possib
----- diversity: 0.5
----- Generating with seed: "r word the same. (and countless more tim"
r word the same. (and countless more times, and the philosopher's reason is an exact don't as for his one, it is a to common sense of actual respects of comparison is understood comparison that counterpart of the ersatz worlds according to a possible worl

the problem of this-worldly theory, that ersatz world is having that different also the ordinary presing the worlds are the ways the concrete world allonds is instantiated. 
() where the way that world we have a sense in any concert world can be classion, then it so that the subject reasons have the first e
----- diversity: 0.5
----- Generating with seed: "ue for any speaker is the function that "
ue for any speaker is the function that seems to be confettonate individual can be a donkey, then modal realists of sentences and concrete world we can say that and our concrete world - included plural that its selection is in some truth. i would take the property does not do where the property of being a relation which are in the same planet in the simple world. then he as i am for indesertole. i sue on assignment of such it is many wa
----- diversity: 1.0
----- Generating with seed: "ue for any speaker is the function that "
ue for any speaker is the function that we are goods. one world is

 hope. maybe ours is one of those big world? i say they intece only if any of these worlds, but tales to respection does s uned universals and croig by it also since to prinot sharing how, thereby endure modality, to or all things: assic individuals, is not that there are the extreme exist runce, or distinction dipfeating labser, by incliniduil distinct things (make by wuthtuagen to. hendehwy sy the head onpetevewit, i noweven bet at le
Epoch 182/200
259976/259976 [==============================] - 141s 542us/step - loss: 0.8114

----- Generating text after Epoch: 181
----- diversity: 0.2
----- Generating with seed: "eterminacy - seems best. (at least it le"
eterminacy - seems best. (at least it leadl, which are the properties are allogical contents in the set of this to composition of properties are all the worlds. the properties of what the worlds the one that the worlds where the concrete world is composed of worlds. we niture of a rest of the single world. it is not an and a world 

something such donkeys, holrs arifain so. there are a single function, who wiser domine indabitation with dotholus have the abuned two seems representing as a great tratestentiogive and similarity of eqriving inerty is constituing of sting domains. but if it mught mi
Epoch 189/200
259976/259976 [==============================] - 156s 601us/step - loss: 0.8133

----- Generating text after Epoch: 188
----- diversity: 0.2
----- Generating with seed: "are not very many candidates, since they"
are not very many candidates, since they may be content to the concrete world is the satisfies that worlds there are speaking and of the own predicate related that the concrete world is instantiated the sentence things of the properties are the way to say that the same thing expression of the particles is the part. i think the things that we have no problem. therefore the concrete world they are anti-haecceitistic different worlds, in s
----- diversity: 0.5
----- Generating with seed: "are not very ma

KeyboardInterrupt: 

In [174]:
#model.save("lewis_bd.h5") # loss = .9667 after like 200 epochs?

In [106]:
model.save("lewis_clean_char.h5") # loss = 0.8142 after about 900 epochs 

In [17]:
def gentxt(seed, length = 400, diversity = .8):
    if len(seed) > 40:
        seed = seed[:40]
    elif len(seed) < 40:
        pad = " " * (40 - len(seed))
        seed = pad + seed
    sentence = seed
    generated = seed
    sys.stdout.write(seed)
    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [19]:
def gentxt2(seed, length = 400, diversity = .8):
    if len(seed) > 40:
        seed = seed[:40]
    elif len(seed) < 40:
        pad = " " * (40 - len(seed))
        seed = pad + seed
    sentence = seed
    generated = seed
    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    print(generated)

In [102]:
gentxt2("the problem with concrete possible worlds is", length = 400, diversity = .3)

the problem with concrete possible worlds that are form of the other worlds. it is a way that a world where the same counterpart of the set of things may be contingence many respects of others and a property partly is the same relations, the counterpart relations that differing on a relativial one of them to a counterpart relation that many of the parts of the other worlds are abstractions whereby the out to be ours. but if there is a w


In [103]:
wts = model.get_weights()
len(wts)

8

In [104]:
for wt_idx in range(len(wts)):
    print("Weight #{}: {}".format(wt_idx, wts[wt_idx].shape))
    print("Min/Max weights", np.max(wts[wt_idx]), np.min(wts[wt_idx]))
    print("Close to zero:", np.sum(np.isclose(wts[wt_idx], 0)))
    print()

Weight #0: (48, 384)
Min/Max weights 8.571241 -9.619392
Close to zero: 0

Weight #1: (128, 384)
Min/Max weights 2.1191409 -1.9466166
Close to zero: 0

Weight #2: (384,)
Min/Max weights 0.43705904 -2.6574779
Close to zero: 0

Weight #3: (48, 384)
Min/Max weights 9.405673 -7.0403075
Close to zero: 0

Weight #4: (128, 384)
Min/Max weights 2.92442 -3.4110582
Close to zero: 0

Weight #5: (384,)
Min/Max weights 1.605154 -1.6481296
Close to zero: 0

Weight #6: (256, 48)
Min/Max weights 7.021394 -6.032216
Close to zero: 0

Weight #7: (48,)
Min/Max weights 1.1436297 -10.604793
Close to zero: 0



In [131]:
from keras.models import load_model

t2 = load_model('lewis_bd.h5')

In [146]:
sent = "the problem with concrete possible world"
sent_vecs = np.zeros((1, 40, len(chars)))
for t, char in enumerate(sent):
    sent_vecs[0, t, char_indices[char]] = 1.
preds = t2.predict(sent_vecs, verbose=0)[0]
next_char = indices_char[np.random.choice(range(len(chars)), p = preds)]

To bundle:
+ model
+ char_indices
+ indices_char
+ chars? (could be rebuilt from above)


## Part 2: word-by-word

In [61]:
p = re.compile(r"\d")

with open("C:/Users/bkrak_000/Downloads/plurality.txt", "r", encoding="utf-8", errors="replace") as f:
    text = f.read().lower()
    text = p.sub('', text)
text_wds = text.split()
print ("corpus length = {} chars, {} words.".format(len(text), len(text_wds)))



corpus length = 779966 chars, 133344 words.


In [84]:
from gensim.models import Word2Vec
w2v = Word2Vec([text_wds], size=100, window=5, workers=4)

In [91]:
w2v.wv.most_similar('donkey')

[('simply', 0.9245911836624146),
 ('unified', 0.9238752126693726),
 ('imagine', 0.9230960607528687),
 ('indeed,', 0.922924280166626),
 ('diamonds', 0.9229161739349365),
 ('universal', 0.9221081733703613),
 ('since', 0.921993613243103),
 ('account', 0.9218493700027466),
 ('up', 0.9218412637710571),
 ('class', 0.921790599822998)]

In [63]:
# cut the text in semi-redundant sequences of maxlen words
maxlen_w = 10
step = 3
# sentences = overlapping sequence of characters of len 10
sentences = []
# next_chars = the next word for every sentence
next_words = []
for i in range(0, len(text_wds) - maxlen_w, step):
    sentences.append(text_wds[i: i + maxlen_w])
    next_words.append(text_wds[i + maxlen_w])
print('nb sequences:', len(sentences))

nb sequences: 44445


In [64]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen_w, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1

Vectorization...


In [65]:
print('Build model...')
model2 = Sequential()
model2.add(Bidirectional(GRU(128), input_shape=(maxlen_w, len(words))))
model2.add(Dropout(.3))
model2.add(Dense(len(words)))
model2.add(Activation('softmax'))

optimizer = Adam(lr=0.001, clipnorm = 1.)
model2.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [66]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text_wds) - maxlen_w - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text_wds[start_index: start_index + maxlen_w]
        generated += " ".join(sentence)
        print('----- Generating with seed: "' + generated + '"')
        sys.stdout.write(generated)

        for i in range(50):
            x_pred = np.zeros((1, maxlen_w, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model2.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            sys.stdout.write(" " + next_word)
            sys.stdout.flush()
            generated += next_word
            sentence.append(next_word)
            sentence = sentence[1:]
        print()

In [67]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model2.fit(x, y,
          batch_size=128, epochs=200,
          callbacks=[print_callback])

Epoch 1/200
44445/44445 [==============================] - 56s 1ms/step - loss: 7.0467

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "is, it must be a consistent set whose consistency would"
is, it must be a consistent set whose consistency would the the the of we that the of (), that the of the peculiarly that that the of of are actual the the any of a homo the dissimilarity. telescope. and in of of the yes the given the approximations the unsatisfactory of believe the the problem, the bernard; of
----- diversity: 0.5
----- Generating with seed: "is, it must be a consistent set whose consistency would"
is, it must be a consistent set whose consistency would is a between the that a and concrete of does world of - we the possible of the instance, of and is for - to the where not all. as is of various sort it. to is for a might are the speaking the , all of is for the
----- diversity: 1.0
----- Generating with seed: "is, it must be a consistent se

. now i can return to a piece of unfinished being. degree, of i have the real element i do it said in well to haecceitism concerning quite a non- truth, as of section meaning we boot. in the universal has to say that a ersatz world. or only by qua time, if that is whole to make a possibility
----- diversity: 1.2
----- Generating with seed: ". now i can return to a piece of unfinished"
. now i can return to a piece of unfinished good appearance, more, nine other-worldly speaker-relativity ersatz as is only actualised composed the example take different objection overlap the consistency if another case, similar that one of representation acquainted de then there is no no good such tolerated, serious face sole context with how the way of section spatiotemporal still
Epoch 10/200
44445/44445 [==============================] - 52s 1ms/step - loss: 3.8326

----- Generating text after Epoch: 9
----- diversity: 0.2
----- Generating with seed: "that establishes a sense in which we may say that"

44445/44445 [==============================] - 52s 1ms/step - loss: 2.6848

----- Generating text after Epoch: 13
----- diversity: 0.2
----- Generating with seed: "are, all but one of the worlds and the great"
are, all but one of the worlds and the great western as a basic western which the whole of belief and desire as an sake of being par with an sense if it would be a good reason to understand what we are to be correct. to what it does not at a world where the dragon world w de
----- diversity: 0.5
----- Generating with seed: "are, all but one of the worlds and the great"
are, all but one of the worlds and the great friends noel' variable; of recombination to be a choice of unproblematic. i am not less than the same sort that is, as follows. the first world or its non-spatiotemporal one - one and thing - for the second modifier head question. as less than my members. for a certain sentence
----- diversity: 1.0
----- Generating with seed: "are, all but one of the worlds and the great

be true together. the second need for primitive modality comes here, to serve that be a contingent city and we are a settled of semantic values for the metaphysics bad in this taken unshifted. given (at as much just so but if we version as the duplicate theory might be unified as requires explicitly i want to help predicates are
----- diversity: 1.2
----- Generating with seed: "be true together. the second need for primitive modality comes"
be true together. the second need for primitive modality comes circularity.) individuals; at of a structural system of question is a victim if we don't had can will bigger a different ersatz laws represent so therefore we would say more. what - things we must count with the first such excluding half the razor. how and we can say it,
Epoch 23/200
44445/44445 [==============================] - 53s 1ms/step - loss: 0.8548

----- Generating text after Epoch: 22
----- diversity: 0.2
----- Generating with seed: "i had not fully appreciated the difference 

you make a second mystery, because you don't tell me which total being the possibility of it. there is no central than the plural kaplan, on confusions sometimes continuum-sized leave into for some to explicit his start thus different particles might be present here in one impossible difference in the sort of comparison. your predicate particulars one basic of properties,
Epoch 31/200
44445/44445 [==============================] - 51s 1ms/step - loss: 0.2684

----- Generating text after Epoch: 30
----- diversity: 0.2
----- Generating with seed: "for any speaker whatever; not so for the first. so"
for any speaker whatever; not so for the first. so a counterpart world selects genuine concrete world, then then is a possible possible world - can be a possible way of all as my single world. we is supposed to distinguish them from the worlds taken taken might be a set of its divisors. individuals that any proper description iff
----- diversity: 0.5
----- Generating with seed: "for any speake

...


----- Generating text after Epoch: 164
----- diversity: 0.2
----- Generating with seed: "may well be a problem. if we had two elements"
may well be a problem. if we had two elements is exactly at least gone closest world is so that is; - such possibilities composed qualitatively now as the semantic values i find that assigns at all some along their recurrence, with the set of universals, - for the word let of say that the particular property to be a
----- diversity: 0.5
----- Generating with seed: "may well be a problem. if we had two elements"
may well be a problem. if we had two elements is exactly at least gone has a donkey talks iff three selected iff which the whole has a counterpart of a richer ersatz world that is, then up you exactly depends whether such points and desire i don't should replace such something to at least our world. false anything will
----- diversity: 1.0
----- Generating with seed: "may well be a problem. if we had two elements"
may well be a problem. if we h

that keep silent about which frames, if any, afford correct together. we we are solar charge properties two matters are their infinite end or by indiscernible one natures - concerns alternatives - just do just do how they have been mind true at some of versus single properties, we are not on the mereological physical. of recombination; are are merge.
----- diversity: 1.2
----- Generating with seed: "that keep silent about which frames, if any, afford correct"
that keep silent about which frames, if any, afford correct restrictedly, we we are them thinking our indexing; re. they they are � in as explicitly way that way. grant mean, you know now as is so so the intrinsic instance of worlds are not the pair would at have to be contingent central cause the past there are all
Epoch 174/200
44445/44445 [==============================] - 51s 1ms/step - loss: 0.0073

----- Generating text after Epoch: 173
----- diversity: 0.2
----- Generating with seed: "at countless possible worlds. it's just

de re concerning the named individuals. suppose again that ours is one of nature i them. said that it us so they - on turn de re or if to get discredit. trans-world identity divided by theory, may even do not to quantify what what haecceitistic account is not to be right than the right content that 'at time is
----- diversity: 0.5
----- Generating with seed: "de re concerning the named individuals. suppose again that ours"
de re concerning the named individuals. suppose again that ours is one of nature i am (). more less once rather than my questions he knew with his knowledge. - for the preferred world is not the other sort. clearly, associated my fact that is not what what it very separate according is exactly de like if means of wins;
----- diversity: 1.0
----- Generating with seed: "de re concerning the named individuals. suppose again that ours"
de re concerning the named individuals. suppose again that ours is one of nature i am very seen to (if and perhaps themselves being what 

all very well for me, since i believe in arbitrary by causal worlds. but there are not. for any pattern we even just as it sometimes he well out the same world is, a non-identity linguistic clean but that require return he - two special to introduce them those parts. when we are different possible worlds, they are limited relative
----- diversity: 1.0
----- Generating with seed: "all very well for me, since i believe in arbitrary"
all very well for me, since i believe in arbitrary by causal worlds. such some special for instance there must be taken taken present a counterpart according to a certain system of belief different semantic values values values themselves well be. both values constants themselves put p' you mean come come well interpreted with nothing about () except domains because
----- diversity: 1.2
----- Generating with seed: "all very well for me, since i believe in arbitrary"
all very well for me, since i believe in arbitrary by causal worlds. i actually cannot believe

only intrinsic properties of a thing are those it has a certain time like a certain system to g iff we coexist in essential to every distance that require standpoint of humphrey two duplicate elements; together that. it is, is, would are be at noah and the realms represent rest.) whatever - - must happen identified so here intrinsically ()
Epoch 195/200
44445/44445 [==============================] - 51s 1ms/step - loss: 0.0044

----- Generating text after Epoch: 194
----- diversity: 0.2
----- Generating with seed: "be an anti-haecceitist in more or less the sense that"
be an anti-haecceitist in more or less the sense that makes so, does not at least because of it, when we speak inconstant. which modal realism is that is so that and but there are supposed to be looking in the same theory has somehow evokes and the proton, have the same instances; part of other other parts between property
----- diversity: 0.5
----- Generating with seed: "be an anti-haecceitist in more or less the sense 

In [68]:
model2.save("lewis_clean_word.h5") #loss = .0072, 225 epochs